* intro to kinesis
* configure your AWS credentials
* KCL wordputter
* implementing simple processor class (echo)
* implementing simple counter (no ordering)
* testing with two KCL wordputters with lag (simulating long network delay)
* implement counter with buffer

# Intro

At Sqreen we are using AWS Kinesis service to process data from our agents in near real-time.

## Requirements

To install dependencies, run the following commands at the command line (i.e. in the shell).
    
```
pip install aws
pip install boto
```

## Configure AWS credentials

To connect to AWS, you must first create your credentials (you will get them from the AWS Console). Then, simply configure them using the following command:

```aws configure --profile blogpost-kinesis```

`blogpost-kinesis` is the name of the profile you will use for this tutorial. You will need to copy you public and secret access keys obtained from AWS Management Console.

## Creating stream


Let's create our first stream. You can either do it using the AWS Console or the API. We will use the second approach. First, we need to define the name of the stream, the region in which we will create it, and the profile to use for our AWS credentials (you can set it to `None` if you use the default profile).

In [1]:
stream_name =  'blogpost-word-stream'
region = 'eu-west-1'
aws_profile = 'blogpost-kinesis'

Now we can use `boto` library to create the stream and wait until it becomes active.

In [2]:
import boto
from boto.kinesis.exceptions import ResourceInUseException
import os
import time

if aws_profile:
    os.environ['AWS_PROFILE'] = aws_profile

# connect to the kinesis
kinesis = boto.kinesis.connect_to_region(region)

try:
    # create the stream
    kinesis.create_stream(stream_name, 1)
    print('stream {} created in region {}'.format(stream_name, region))
except ResourceInUseException:
    print('stream {} already exists in region {}'.format(stream_name, region))

def get_status():
    r = kinesis.describe_stream(stream_name)
    description = r.get('StreamDescription')
    status = description.get('StreamStatus')
    return status

# wait for the stream to become active
while get_status() != 'ACTIVE':
    time.sleep(1)
print('stream {} is active'.format(stream_name))

stream blogpost-word-stream already exists in region eu-west-1
stream blogpost-word-stream is active


# Putting data into streams

To have operational stream processing, we need a source of the messagers (producer in AWS terminology) and receiver (consumer) that will obtain and process the messages. We will first define the producer.

In [3]:
import datetime
import time
import threading
from boto.kinesis.exceptions import ResourceNotFoundException

class KinesisProducer(threading.Thread):
    """Producer class for AWS Kinesis streams
    
    This class will emit records with the IP addresses as partition key and
    the emission timestamps as data"""
    
    def __init__(self, stream_name, sleep_interval=None, ip_addr='8.8.8.8'):
        self.stream_name = stream_name
        self.sleep_interval = sleep_interval
        self.ip_addr = ip_addr
        super().__init__()
        
    def put_record(self):
        """put a single record to the stream"""
        timestamp = datetime.datetime.utcnow()
        part_key = self.ip_addr
        data = timestamp.isoformat()

        kinesis.put_record(self.stream_name, data, part_key)
    
    def run_continously(self):
        """put a record at regular intervals"""
        while True:
            self.put_record()
            time.sleep(self.sleep_interval)
                
    def run(self):
        try:
            if self.sleep_interval:
                self.run_continously()
            else:
                self.put_record()
        except ResourceNotFoundException:
            print('stream {} not found. Exiting'.format(self.stream_name))

Note that for the partion key we used the IP address and for the data the timestamps. In theory, you are almost completely free to choose whatever you want for the data, as long as it's less than 50 KB of size. If you need emit larger data, you need to split it into several records. The partion key must be a string shorter than 256 characters, it will be used to determine which shard to send the data to (see below).

Note that we implemented the KinesisProducer as a python thread, such that it can run in the background and won't block Python REPL.

Now we create two of such producers with different IP addresses and different intervals between emitting the consecutive messages.

In [4]:
producer1 = KinesisProducer(stream_name, sleep_interval=2, ip_addr='8.8.8.8')
producer2 = KinesisProducer(stream_name, sleep_interval=5, ip_addr='8.8.8.9')
producer1.start()
producer2.start()

# Consuming from stream

Consumers receive the messages from the stream and process them. Their output could be messages forwared to another stream, file saved in the filesystem (or Amazon S3 storage) or records stored in a database. 

In [5]:
from boto.kinesis.exceptions import ProvisionedThroughputExceededException
import datetime

In [6]:
# https://github.com/aws-samples/kinesis-poster-worker/blob/master/worker.py

class KinesisWorker:
    """Generic Consumer for Amazon Kinesis Streams"""
    def __init__(self, stream_name, shard_id, iterator_type,
                 worker_time=30, sleep_interval=0.5):
   
        self.stream_name = stream_name
        self.shard_id = str(shard_id)
        self.iterator_type = iterator_type
        self.worker_time = worker_time
        self.sleep_interval = sleep_interval
        
    def process_records(self, records):
        """the main logic of the Consumer that needs to be implemented"""
        raise NotImplementedError
    
    @staticmethod
    def iter_records(records):
        for record in records:
            part_key = record['PartitionKey']
            data = record['Data']
            yield part_key, data
    
    def run(self):
        """poll stream for new records and forward them to process_records method"""
        response = kinesis.get_shard_iterator(self.stream_name,
            self.shard_id, self.iterator_type)
        
        next_iterator = response['ShardIterator']

        start = datetime.datetime.now()
        finish = start + datetime.timedelta(seconds=self.worker_time)
        
        while finish > datetime.datetime.now():
            try:
                response = kinesis.get_records(next_iterator, limit=25)
        

                records = response['Records']
            
                if records:
                    self.process_records(records)
            
                next_iterator = response['NextShardIterator']
                time.sleep(self.sleep_interval)
            except ProvisionedThroughputExceededException as ptee:
                time.sleep(1)

Note that each stream can have many consumers that receive all the messages and process them independently. Now, we will implement `process_records` method that will simply print the received messages to the `stdout`.

In [7]:
class EchoWorker(KinesisWorker):
    """Consumers that echos received data to standard output"""
    def process_records(self, records):
        """print the partion key and data of each incoming record"""
        for part_key, data in self.iter_records(records):
            print(part_key, ":", data)

We run the consumer on our stream. Note that we need to pass the shard ID and the position in the stream to start processing the messages. For the later, we can choose between that newest (`LATEST`) or the oldest (`TRIM_HORIZON`) record in the stream. 

The streams are partitioned into seperate "sub-streams" (called shards) that receive messages from the same source. The target shard for each message is determined from the partion key. Each consumer can read from one or more shards, but there must be at least one consumer per shard, otherwie some messages will be lost. Since, we only use one shard in this example, we can directly pass the default shard ID. If you need to configure more than one shard (to increase the throughput), you will need to query the stream for the IDs of all active shards using the API. For the sake of this tutorial, we will assume that we have only a single shard.

In [8]:
shard_id = 'shardId-000000000000'
iterator_type =  'LATEST'
worker = EchoWorker(stream_name, shard_id, iterator_type, worker_time=10)

In [9]:
worker.run()

8.8.8.8 : 2018-07-29T22:35:46.694763
8.8.8.8 : 2018-07-29T22:35:48.855009
8.8.8.9 : 2018-07-29T22:35:49.951666
8.8.8.8 : 2018-07-29T22:35:51.015702
8.8.8.8 : 2018-07-29T22:35:53.134880


As expected the consumer printed all received records with their partion keys (IP addresses) and data (timestamps). 

In [10]:
from collections import defaultdict, Counter
from dateutil import parser
from operator import itemgetter

class CounterWorker(KinesisWorker):
    """Consumer that counts IP occurences in 1-minute time buckets"""
    
    def __init__(self, stream_name, shard_id, iterator_type, worker_time):
        self.time_buckets = defaultdict(Counter)
        sleep_interval = 20 # seconds
        super().__init__(stream_name, shard_id, iterator_type, worker_time, sleep_interval)
        
    def print_counters(self):
        """helper method to show counting results"""
        
        now = datetime.datetime.utcnow()
        print("##### Last run at {}".format(now))
        for timestamp, ip_counts in self.time_buckets.items():
            # sort counts with respect to the IP address
            ip_counts = sorted(ip_counts.items(), key=itemgetter(0))
            print(timestamp, ':', list(ip_counts))
            
    def process_records(self, records):
        for ip_addr, timestamp_str in self.iter_records(records):
            timestamp = parser.parse(timestamp_str)
            timestamp = timestamp.replace(second=0, microsecond=0)
            self.time_buckets[timestamp][ip_addr] += 1
        self.print_counters()         

In [11]:
worker = CounterWorker(stream_name, shard_id, iterator_type, worker_time=120)
worker.run()

##### Last run at 2018-07-29 22:36:16.020524
2018-07-29 22:35:00 : [('8.8.8.8', 2)]
2018-07-29 22:36:00 : [('8.8.8.8', 7), ('8.8.8.9', 4)]
##### Last run at 2018-07-29 22:36:36.202270
2018-07-29 22:35:00 : [('8.8.8.8', 2)]
2018-07-29 22:36:00 : [('8.8.8.8', 16), ('8.8.8.9', 7)]
##### Last run at 2018-07-29 22:36:56.384823
2018-07-29 22:35:00 : [('8.8.8.8', 2)]
2018-07-29 22:36:00 : [('8.8.8.8', 26), ('8.8.8.9', 11)]
##### Last run at 2018-07-29 22:37:16.581523
2018-07-29 22:35:00 : [('8.8.8.8', 2)]
2018-07-29 22:36:00 : [('8.8.8.8', 27), ('8.8.8.9', 12)]
2018-07-29 22:37:00 : [('8.8.8.8', 8), ('8.8.8.9', 3)]
##### Last run at 2018-07-29 22:37:36.753725
2018-07-29 22:35:00 : [('8.8.8.8', 2)]
2018-07-29 22:36:00 : [('8.8.8.8', 27), ('8.8.8.9', 12)]
2018-07-29 22:37:00 : [('8.8.8.8', 18), ('8.8.8.9', 7)]


In [12]:
# delete the stream at the end of the exercise to minimize AWS costs
kinesis.delete_stream(stream_name)

stream blogpost-word-stream not found. Exiting
stream blogpost-word-stream not found. Exiting
